In [1]:
pip install requests beautifulsoup4 


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from bs4 import BeautifulSoup

def search_amazon(product):
    
    product = product.replace(' ', '+')

    
    url = f"https://www.amazon.in/s?k={product}"

    
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)

    
    soup = BeautifulSoup(response.content, 'html.parser')


    products = soup.find_all('div', {'data-component-type': 's-search-result'})

    
    for product in products:
        title = product.h2.text
        link = 'https://www.amazon.in' + product.h2.a['href']
        try:
            price = product.find('span', 'a-price').find('span', 'a-offscreen').text
        except AttributeError:
            price = "Price not available"
        print(f"Product: {title}, Price: {price}, Link: {link}")


product_to_search = input("Enter the product you want to search for: ")
search_amazon(product_to_search)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time


def scrape_product_details(search_query):
    
    driver = webdriver.Chrome()
    driver.get('https://www.example.com/')  
    
    
    search_box = driver.find_element_by_name('search')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.ENTER)
    
    
    time.sleep(5)
    
    
    products_list = []
    
    
    for page in range(1, 4):
        
        products = driver.find_elements_by_css_selector('div.product')  
        
        for product in products:
            
            brand_name = product.find_element_by_css_selector('...').text or "-"
            product_name = product.find_element_by_css_selector('...').text or "-"
            price = product.find_element_by_css_selector('...').text or "-"
            return_exchange = product.find_element_by_css_selector('...').text or "-"
            expected_delivery = product.find_element_by_css_selector('...').text or "-"
            availability = product.find_element_by_css_selector('...').text or "-"
            product_url = product.find_element_by_css_selector('a').get_attribute('href') or "-"
            
            
            products_list.append({
                "Brand Name": brand_name,
                "Name of the Product": product_name,
                "Price": price,
                "Return/Exchange": return_exchange,
                "Expected Delivery": expected_delivery,
                "Availability": availability,
                "Product URL": product_url
            })
        
        
        next_page_button = driver.find_element_by_css_selector('...')  
        if next_page_button:
            next_page_button.click()
            time.sleep(5)
        else:
            break
    
    driver.quit()
    
    
    df = pd.DataFrame(products_list)
    df.to_csv(f'{search_query}_products.csv', index=False)
    
scrape_product_details('example search query') 


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os


def fetch_images(search_query, number_of_images):
    
    driver = webdriver.Chrome()
    driver.get('https://images.google.com/')
    
    
    search_box = driver.find_element_by_name('q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.ENTER)
    
    
    time.sleep(5)
    
    
    images = driver.find_elements_by_css_selector('img.rg_i.Q4LuWd')
    
    
    if not os.path.exists(search_query):
        os.makedirs(search_query)
    
    
    for i in range(number_of_images):
        try:
            
            images[i].click()
            time.sleep(2)
            
            
            large_image = driver.find_elements_by_css_selector('img.n3VNCb')[1]
            image_url = large_image.get_attribute('src')
            
            
            response = requests.get(image_url)
            with open(f'{search_query}/{search_query}_{i+1}.jpg', 'wb') as file:
                file.write(response.content)
        except Exception as e:
            print(f"Error - {e}")
    
    driver.quit()


keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
for keyword in keywords:
    fetch_images(keyword, 10) 


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_flipkart(search_query):
    
    url = f"https://www.flipkart.com/search?q={search_query}"
    
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    product_containers = soup.find_all("div", class_="_1AtVbE")

    
    products = []

    for container in product_containers:
        try:
           
            name_info = container.find("div", class_="_4rR01T")
            if name_info:
                name = name_info.text.split(" ", 1)
                brand_name = name[0]
                smartphone_name = name[1]
            else:
                brand_name = "-"
                smartphone_name = "-"
            
            # Product URL
            product_url_tag = container.find("a", class_="_1fQZEK")
            product_url = "https://www.flipkart.com" + product_url_tag['href'] if product_url_tag else "-"
            
            # Price
            price_tag = container.find("div", class_="_30jeq3 _1_WHN1")
            price = price_tag.text if price_tag else "-"
            
            # Features
            features = container.find_all("li", class_="rgWa7D")
            feature_dict = {
                "Colour": "-",
                "RAM": "-",
                "Storage(ROM)": "-",
                "Primary Camera": "-",
                "Secondary Camera": "-",
                "Display Size": "-",
                "Battery Capacity": "-"
            }
            for feature in features:
                feature_text = feature.text
                if "RAM" in feature_text and "ROM" in feature_text:
                    ram, rom = feature_text.split("|")
                    feature_dict["RAM"] = ram.strip()
                    feature_dict["Storage(ROM)"] = rom.strip()
                elif "Display" in feature_text:
                    feature_dict["Display Size"] = feature_text
                elif "Camera" in feature_text and "Primary" not in feature_dict["Primary Camera"]:
                    feature_dict["Primary Camera"] = feature_text
                elif "Battery" in feature_text:
                    feature_dict["Battery Capacity"] = feature_text
                else:
                    # Check if the feature is one of the secondary camera features
                    if "Secondary Camera" not in feature_dict["Secondary Camera"]:
                        feature_dict["Secondary Camera"] = feature_text

            
            products.append({
                "Brand Name": brand_name,
                "Smartphone name": smartphone_name,
                "Colour": feature_dict["Colour"],
                "RAM": feature_dict["RAM"],
                "Storage(ROM)": feature_dict["Storage(ROM)"],
                "Primary Camera": feature_dict["Primary Camera"],
                "Secondary Camera": feature_dict["Secondary Camera"],
                "Display Size": feature_dict["Display Size"],
                "Battery Capacity": feature_dict["Battery Capacity"],
                "Price": price,
                "Product URL": product_url
            })
        except Exception as e:
    
            print(f"Error processing product: {e}")
            continue

    
    df = pd.DataFrame(products)
    
    
    df.to_csv("flipkart_smartphones.csv", index=False)
    print("Data saved to flipkart_smartphones.csv")


scrape_flipkart("Oneplus Nord")


In [ ]:
from geopy.geocoders import Nominatim

def get_geospatial_coordinates(city_name):
    
    geolocator = Nominatim(user_agent="geoapiExercises")
    

    location = geolocator.geocode(city_name)
    
    if location:
        return location.latitude, location.longitude
    else:
        return None, None


city_name = "San Francisco"
latitude, longitude = get_geospatial_coordinates(city_name)

if latitude and longitude:
    print(f"The latitude and longitude of {city_name} are {latitude}, {longitude}.")
else:
    print(f"Coordinates for {city_name} could not be found.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_digit_gaming_laptops():
    
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    laptop_containers = soup.find_all("div", class_="TopNumbeHeading sticky-footer")

    
    laptops = []

    for container in laptop_containers:
        try:
            
            name_tag = container.find("div", class_="left_side")
            name = name_tag.h3.text if name_tag else "-"
            
            
            details = container.find_all("div", class_="Specs-list")
            detail_dict = {
                "Laptop Name": name,
                "Processor": "-",
                "Graphics Processor": "-",
                "RAM": "-",
                "Screen Size": "-",
                "Resolution": "-",
                "Weight": "-",
                "Price": "-"
            }
            for detail in details:
                detail_text = detail.text.split(':')
                if len(detail_text) == 2:
                    key, value = detail_text
                    detail_dict[key.strip()] = value.strip()
            
            
            laptops.append(detail_dict)
        except Exception as e:
            
            print(f"Error processing laptop: {e}")
            continue

    
    df = pd.DataFrame(laptops)
    

    df.to_csv("digit_gaming_laptops.csv", index=False)
    print("Data saved to digit_gaming_laptops.csv")


scrape_digit_gaming_laptops()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_forbes_billionaires():
    
    url = "https://www.forbes.com/billionaires/"
    

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    script = soup.find("script", {"type": "application/json"})
    json_data = script.string
    
    
    import json
    data = json.loads(json_data)
    

    billionaires = data["props"]["pageProps"]["tableData"]

    
    details = []

    for billionaire in billionaires:
        details.append({
            "Rank": billionaire.get("rank", "-"),
            "Name": billionaire.get("personName", "-"),
            "Net worth": billionaire.get("finalWorth", "-"),
            "Age": billionaire.get("age", "-"),
            "Citizenship": billionaire.get("countryOfCitizenship", "-"),
            "Source": billionaire.get("source", "-"),
            "Industry": billionaire.get("industries", "-")
        })

    
    df = pd.DataFrame(details)
    

    df.to_csv("forbes_billionaires.csv", index=False)
    print("Data saved to forbes_billionaires.csv")


scrape_forbes_billionaires() 


In [ ]:
import os
import googleapiclient.discovery
import pandas as pd


def get_youtube_comments(video_id, api_key, max_comments=500):

    youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)

    
    request = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100, 
        textFormat="plainText"
    )
    
    comments = []
    comment_count = 0

    while request and comment_count < max_comments:
        response = request.execute()

        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'Comment': comment['textDisplay'],
                'Upvotes': comment['likeCount'],
                'Published At': comment['publishedAt']
            })
            comment_count += 1

        
        if comment_count >= max_comments:
            break

        
        request = youtube.commentThreads().list_next(request, response)

    
    df = pd.DataFrame(comments)
    return df


if __name__ == "__main__":
    API_KEY = "YOUR_YOUTUBE_API_KEY"
    VIDEO_ID = "YOUR_VIDEO_ID"  
    MAX_COMMENTS = 500

    comments_df = get_youtube_comments(VIDEO_ID, API_KEY, MAX_COMMENTS)
    comments_df.to_csv("youtube_comments.csv", index=False)
    print("Comments saved to youtube_comments.csv")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_hostelworld_london():
    url = "https://www.hostelworld.com/findabed.php/ChosenCity.London/ChosenCountry.England"
    
    # Send the request to hostelworld.com
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all the hostel containers on the page
    hostels = soup.find_all("div", class_="property-card")
    
    # Prepare a list to hold all the hostel details
    hostel_details = []

    for hostel in hostels:
        try:
            
            name = hostel.find("h2", class_="title").text.strip()

        
            distance = hostel.find("span", class_="distance-description").text.strip()

            
            rating_tag = hostel.find("div", class_="score orange big")
            rating = rating_tag.text.strip() if rating_tag else "-"

            
            total_reviews_tag = hostel.find("div", class_="reviews")
            total_reviews = total_reviews_tag.text.strip() if total_reviews_tag else "-"

    
            overall_reviews_tag = hostel.find("div", class_="keyword")
            overall_reviews = overall_reviews_tag.text.strip() if overall_reviews_tag else "-"

            
            privates_price_tag = hostel.find("div", class_="price title-2")
            privates_from_price = privates_price_tag.text.strip() if privates_price_tag else "-"

            dorms_price_tag = hostel.find("div", class_="price dorm")
            dorms_from_price = dorms_price_tag.text.strip() if dorms_price_tag else "-"

            
            facilities_list = hostel.find_all("ul", class_="facility-segment")
            facilities = [facility.text.strip() for facility in facilities_list]

            
            description_tag = hostel.find("div", class_="rating-factors")
            property_description = description_tag.text.strip() if description_tag else "-"

            
            hostel_details.append({
                "Hostel Name": name,
                "Distance from City Centre": distance,
                "Ratings": rating,
                "Total Reviews": total_reviews,
                "Overall Reviews": overall_reviews,
                "Privates From Price": privates_from_price,
                "Dorms From Price": dorms_from_price,
                "Facilities": ", ".join(facilities),
                "Property Description": property_description
            })
        except Exception as e:
            
            print(f"Error processing hostel: {e}")
            continue

    
    df = pd.DataFrame(hostel_details)
    
    
    df.to_csv("hostelworld_london.csv", index=False)
    print("Data saved to hostelworld_london.csv")

    
scrape_hostelworld_london()
